# Working with linear networks

Mink's *layers* work exactly as sklearn *transformers*, and linear networks work analogously to sklearn Pipelines. Below, the usage of mink layers as transformers is illustrated.

## imports

In [1]:
from pprint import pprint

In [2]:
import numpy as np
import tensorflow as tf
from sklearn.datasets import make_classification
from sklearn.preprocessing import LabelBinarizer
from sklearn.pipeline import Pipeline

In [3]:
from mink import layers
from mink import nonlinearities
from mink import objectives
from mink import updates
from mink import make_network

## load data

Use sklearn toy classification data.

In [4]:
X, y = make_classification(
    n_samples=2000,
    n_classes=5,
    n_informative=10,
    random_state=0,
)
y = LabelBinarizer().fit_transform(y)

## prepare a neural network by using sklearn's Pipeline

We define a simple linear network with an input layer, a hidden layer, and an output layer, by passing a list of name/layer tuples to an sklearn `Pipeline`.

In [5]:
network = Pipeline([
    ('input', layers.InputLayer()),
    ('dense-1', layers.DenseLayer()),
    ('dense-2', layers.DenseLayer(
        num_units=5,
        nonlinearity=nonlinearities.Softmax(),
        )),
])

Altnernatively, we may use the `make_network` function from mink, which works analogously to sklearn's `make_pipeline`. This saves us the need to write a name for each layer.

In [6]:
network = make_network([
    layers.InputLayer(),
    layers.DenseLayer(),
    layers.DenseLayer(
        num_units=5,
        nonlinearity=nonlinearities.Softmax(),
    ),
])

In [7]:
pprint(network.steps)

[('input', InputLayer(Xs=None, make_logs=False, name=None, ys=None)),
 ('dense-1',
  DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(value=0.0),
      incoming=InputLayer(Xs=None, make_logs=False, name=None, ys=None),
      make_logs=False, name=None, nonlinearity=None, num_units=None)),
 ('dense-2',
  DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(value=0.0),
      incoming=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(value=0.0),
      incoming=InputLayer(Xs=None, make_logs=False, name=None, ys=None),
      make_logs=False, name=None, nonlinearity=None, num_units=None),
      make_logs=False, name=None, nonlinearity=Softmax(), num_units=5))]


## get symbolic output

First we define the symbolic input.

In [8]:
Xs = tf.placeholder(dtype='float32', shape=(None, 20))

In [9]:
ys = tf.placeholder(dtype='float32', shape=(None, 5))

Like in sklearn, we can call `fit_transform` and the pipeline to get the output.

In [10]:
ys_out = network.fit_transform(Xs, ys)

## fit neural network

In [11]:
batch_size = 64

In [12]:
loss = objectives.CrossEntropy()(ys, ys_out)
train_step = updates.Momentum()(loss)
inputs = [train_step, loss]

In [13]:
init = tf.initialize_all_variables()
losses = []
with tf.Session() as session:
    session.run(init)
    for epoch in range(30):
        losses_epoch = []
        for i in range((X.shape[0] + batch_size - 1) // batch_size):
            Xb = X[i * batch_size:(i + 1) * batch_size]
            yb = y[i * batch_size:(i + 1) * batch_size]

            feed_dict = {Xs: Xb, ys: yb}
            _, loss = session.run(inputs, feed_dict=feed_dict)
            losses_epoch.append(loss)
        losses_mean = np.mean(losses_epoch)
        losses.append(losses_mean)
        print("Loss: {:.4f}".format(losses_mean))

Loss: 1.4433
Loss: 0.9388
Loss: 0.8456
Loss: 0.7814
Loss: 0.7275
Loss: 0.6811
Loss: 0.6404
Loss: 0.6046
Loss: 0.5724
Loss: 0.5443
Loss: 0.5191
Loss: 0.4967
Loss: 0.4761
Loss: 0.4572
Loss: 0.4401
Loss: 0.4237
Loss: 0.4097
Loss: 0.3960
Loss: 0.3839
Loss: 0.3720
Loss: 0.3611
Loss: 0.3510
Loss: 0.3416
Loss: 0.3329
Loss: 0.3247
Loss: 0.3166
Loss: 0.3095
Loss: 0.3031
Loss: 0.2959
Loss: 0.2900


## prepare a neural network by hand

Alternatively, we can feedforward the output of each layer and pass the output to the next layer by hand. This will result in exactly the same network as defined above.

In [14]:
out = layers.InputLayer(Xs, ys).fit_transform(Xs)
out = layers.DenseLayer().fit_transform(out)
out = layers.DenseLayer(num_units=5, nonlinearity=nonlinearities.Softmax()).fit_transform(out)

In [15]:
loss = objectives.CrossEntropy()(ys, out)
train_step = updates.Momentum()(loss)
inputs = [train_step, loss]

In [16]:
init = tf.initialize_all_variables()
losses = []
with tf.Session() as session:
    session.run(init)
    for epoch in range(30):
        losses_epoch = []
        for i in range((X.shape[0] + batch_size - 1) // batch_size):
            Xb = X[i * batch_size:(i + 1) * batch_size]
            yb = y[i * batch_size:(i + 1) * batch_size]

            feed_dict = {Xs: Xb, ys: yb}
            _, loss = session.run(inputs, feed_dict=feed_dict)
            losses_epoch.append(loss)
        losses_mean = np.mean(losses_epoch)
        losses.append(losses_mean)
        print("Loss: {:.4f}".format(losses_mean))

Loss: 1.3519
Loss: 0.9458
Loss: 0.8517
Loss: 0.7857
Loss: 0.7328
Loss: 0.6886
Loss: 0.6500
Loss: 0.6168
Loss: 0.5873
Loss: 0.5616
Loss: 0.5370
Loss: 0.5150
Loss: 0.4949
Loss: 0.4763
Loss: 0.4591
Loss: 0.4434
Loss: 0.4292
Loss: 0.4160
Loss: 0.4036
Loss: 0.3925
Loss: 0.3821
Loss: 0.3726
Loss: 0.3635
Loss: 0.3554
Loss: 0.3469
Loss: 0.3396
Loss: 0.3316
Loss: 0.3246
Loss: 0.3177
Loss: 0.3111
